# Tuning hyperparameters with Keras tuner

## Preparing data

In [ ]:
from google.colab import drive # loads a library to mount your google drive
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import numpy as np

print('Mounting drive...')
drive.mount('/content/drive') 

!ls "/content/drive/My Drive/Colab Notebooks/data/" # shows all files 

print('Reading data...')
data_path = "/content/drive/My Drive/Colab Notebooks/data/yaleExtB_data.npy" # sets the path to the data folder
X = np.load(data_path)
# data = torch.from_numpy(np.load(data_path))

print('Reading target data...')
target_path = "/content/drive/My Drive/Colab Notebooks/data/yaleExtB_target.npy"
target = np.load(target_path)

print('One hot encoding...')
onehot_encoder=OneHotEncoder(sparse_output=False)

reshaped=target.reshape(len(target), 1)

Y=onehot_encoder.fit_transform(reshaped)

print('Train-test split...')

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=0.33, 
                                                    random_state=42)


print('Calculating PCA...')
# PCA 
nof_prin_components = 200  # PARAMETER for optimisation in expereiments
pca = PCA(n_components=nof_prin_components, whiten=True).fit(X_train)
# applies PCA to the train and test images to calculate the principal components
X_train_pca = pca.transform(X_train) 
X_test_pca = pca.transform(X_test)
print('Done preparing data.')

Mounting drive...
Mounted at /content/drive
Tr0.zip  yaleExtB_data.npy  yaleExtB_target.npy
Reading data...
Reading target data...
One hot encoding...
Train-test split...
Calculating PCA...
Done preparing data.


In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
# If keras-tuner library is not available, install it
print('Installing keras-tuner library...')
!pip install -q -U keras-tuner
print('Finished installing keras-tuner.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 KB 7.5 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

## Creating a callback function to build a model

In [ ]:
def model_builder(hp_tuner):

  # Creating a sequential model
  model = keras.Sequential()
  #model.add(keras.layers.Flatten(input_shape=(4)))

  # input layer
  # model.add(keras.layers.Flatten())   
  model.add(keras.layers.Dense(200, activation='relu'))

  # tuner parameter for search
  # Let's start hidden layer nodes from 64 then go up to 256 with increments 
  # of 16
  tuner_units = hp_tuner.Int('units', min_value=64, max_value=256, step=16)
  model.add(keras.layers.Dense(units=tuner_units, activation='relu'))

  # model.add(keras.layers.Dropout(0.2))

  # output layer
  model.add(keras.layers.Dense(30, activation='softmax'))

  # let's define range of learning rate
  tuner_learning_rate = hp_tuner.Choice('learning_rate', values = [0.1, 0.01, 0.001])
  optimizer = keras.optimizers.Adam(learning_rate=tuner_learning_rate)

  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

## Instantiate the tuner and perform hypertuning

There are four tuners available: RandomSearch, Hyperband, BayesianOptimization, Sklearn

In [ ]:
tuner = kt.Hyperband(model_builder, 
                     objective='val_accuracy', 
                     max_epochs = 10)
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 16, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.01, 0.001], 'ordered': True}


In [ ]:
# Create a callback to stop training when certain validation loss is reached
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Trial 30 Complete [00h 00m 06s]
val_accuracy: 0.049751244485378265

Best val_accuracy So Far: 0.7711442708969116
Total elapsed time: 00h 02m 45s


In [ ]:
# Search for best parameters
tuner.search(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

In [ ]:
# Get the best parameters
best_parameters = tuner.get_best_hyperparameters()[0]

print(f"Best no. of hidden nodes: {best_parameters.get('units')}")
print(f"Best learnig rate: {best_parameters.get('learning_rate')}")

## Build a mode with optimal hyperparameters then train the model

In [ ]:
# Let's train the new model for 50 epochs
model = tuner.hypermodel.build(best_parameters)
history = model.fit(X_train_pca, y_train, epochs=50, validation_split=0.2)

In [ ]:
# Obtain the best epoch
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

## Create a final model with the best parameters and best epoch

In [ ]:
# Create a new model and train for the best epoch
hypermodel = tuner.hypermodel.build(best_parameters)
hypermodel.fit(X_train_pca, y_train, epochs=best_epoch, validation_split=0.2)

In [17]:
# Test the final model with test data then obtain accuracy
eval_result = hypermodel.evaluate(X_test_pca, y_test)
print("[test loss, test accuracy]:", eval_result)

16/16 [==============================] - 0s 3ms/step - loss: 0.2832 - accuracy: 0.9293
[test loss, test accuracy]: [0.2831777334213257, 0.9292929172515869]
